# KV runbook

In [ ]:
%%bash
docker run --name kv -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 postgres:16.2
sleep 2

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE IF NOT EXISTS document (
    id BIGINT PRIMARY KEY,
    data BYTEA
);

"

In [ ]:
import psycopg
from psycopg.abc import Query

def insert(key: int, value: bytes) -> tuple[int, bytes]:
    with psycopg.connect("host=localhost dbname=postgres user=postgres") as conn:
        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO document (id, data)
                VALUES (%s, %s)
                RETURNING *
                """, [key, value])
            return cur.fetchone()


def query(sql: Query) -> list[tuple[int, bytes]]:
    with psycopg.connect("host=localhost dbname=postgres user=postgres") as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            return cur.fetchall()

In [ ]:
data = 'My string'

insert(1, data.encode())


In [ ]:
# Get all documents that has a fish as a pet

key, value = query("""
    SELECT *
    FROM document
    WHERE id = 1
""")[0]

value.decode()

In [ ]:
from IPython.display import Image

Image(filename='postgres.png') 


In [ ]:
with open('./postgres.png', 'rb') as f:
    insert(2, f.read())

key, value = query("""
    SELECT *
    FROM document
    WHERE id = 2
""")[0]


from IPython.display import Image
Image(value) 


In [ ]:
%%bash
docker stop kv
docker rm kv